# Постановка задачи.
Необходимо подготовить данные и обучить модель отпределять дефолтных клиентов.

# 0. Стартовая подготовка.

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    auc,
    roc_auc_score,
    roc_curve,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option("max_column", None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 13

In [4]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [5]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
df_train = pd.read_csv(DATA_DIR+'train.csv')
df_test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

# Вспомогательные функции

In [6]:
# Вывод метрик
def veiw_model(model, y_test, y_pred, size=(11,9)):
    # Вывод типа модели:
    print()
    print('Model Type: ' + str(model))
    print()

    # Вывод confusion matrix:
    sns.set(rc = {'figure.figsize':size})
    sns.set_context(context='paper', font_scale=2, rc=None)
    group_names = ['True\nNegative', 'False\nPositive',
                   'False\nNegative', 'True\nPositive']
    group_counts = [f'{value}' for value in confusion_matrix(y_pred, y_test).flatten()]
    labels = [f'{v1}\n\n{v2}' for v1, v2 in zip(group_names, group_counts)]
    labels = np.asarray(labels).reshape(2, 2)
    ax = sns.heatmap(confusion_matrix(y_pred, y_test),
                     annot=labels, fmt='', cmap='Blues')
    ax.set(xlabel='predicted', ylabel='real', title='Confusion Matrix')
    plt.show()
    print()

def veiw_metrix(y_test, y_pred, y_pred_probs):
    # Вывод значений метрик:
    print('accuracy:', accuracy_score(y_test, y_pred))
    print('precision_score', precision_score(y_test, y_pred))
    print('recall_score:', recall_score(y_test, y_pred))
    print('f1_score:', f1_score(y_test, y_pred))
    
    fpr, tpr, threshold = roc_curve(y_test, y_pred_probs)
    roc_auc = roc_auc_score(y_test, y_pred_probs)
    
    fig, ax = plt.subplots(figsize=(13,7))
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    ax.set_title('Logistic Regression ROC AUC = %0.3f'%roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc='lower right')

# Визуализация числовых данных
def veiw_numeric(column, size = 7, title=None):
    if not title:
        title = column.name
    
    fig, (g1, g2) = plt.subplots(1, 2, figsize = (2*size,size))
    fig.suptitle(f'Histogram and boxplot for {title} ', fontsize=14)
    g1.hist(column, bins = 20, histtype = 'bar', align = 'mid', rwidth = 0.8, color = 'red') # гистограмма
    g2.boxplot(column, vert = False)  # выбросы
    plt.figtext(0.5, 0, title, fontsize = 14, ha='center')
    plt.legend()

# Ищем выбросы
def outliers_iqr(column):
    quartile_1, quartile_3 = np.percentile(column, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    
    new = column.loc[(column < lower_bound) | (column > upper_bound)]    
    
    print(f"{len(new)} выбросов")
    print(f'25-й перцентиль: {quartile_1},', f'75-й перцентиль: {quartile_3},', f"IQR: {iqr}, ", f"Границы выбросов: [{lower_bound}, {upper_bound}].")
    
    return new

# гистограммы логарифмирования
def sqrt_log_veiw(column, a=1):
    # логарифмирование
    log = np.log(column + a)
    veiw_numeric(log, title=f"логарифм {column.name}")

    # взятие квадратного корня
    sqrt = np.sqrt(column)
    veiw_numeric(sqrt, title=f"квадратный корень {column.name}")
    
    # логарифмирование квадратного корня
    log_sqrt = np.log(sqrt + a)
    veiw_numeric(log_sqrt, title=f"логарифм квадратного корня {column.name}")
    
    # взятие квадратного корня логарифма
    sqrt_log = np.sqrt(log)
    veiw_numeric(sqrt_log, title=f"квадратный корень логарифма {column.name}")
    
    return log, sqrt, log_sqrt, sqrt_log
    

# 1. Первичный осмотр данных.

In [7]:
display(df_train.info())
display(df_test.info())

Признак | Описание
--- | ---  
client_id |	идентификатор клиента  
app_date | дата подачи заявки
education |	уровень образования  
sex | пол заёмщика  
age | возраст заёмщика  
car | флаг наличия автомобиля  
car_type | флаг автомобиля-иномарки  
decline_app_cnt | количество отказанных прошлых заявок  
good_work |	флаг наличия «хорошей» работы 
score_bki | скоринговый балл по данным из БКИ
bki_request_cnt | количество запросов в БКИ  
region_rating | рейтинг региона
home_address | категоризатор домашнего адреса  
work_address | категоризатор рабочего адреса  
income | доход заёмщика  
sna | связь заемщика с клиентами банка
first_time| давность наличия информации о заемщике
foreign_passport |	наличие загранпаспорта  
default | наличие дефолта  

In [8]:
display(df_train.head())
display(df_test.head())
display(sample_submission.head())

# Объединим датасеты для обработки.

In [9]:
df_train["sample"] = 0
df_test['default'] = 0
df_test["sample"] = 1
df = df_train.append(df_test, sort=False).reset_index(drop=True)

# Осмотрим пропуски.

In [10]:
df.isnull().sum()

In [11]:
fig, ax = plt.subplots(figsize=(17,13))
sns_heatmap = sns.heatmap(df.isnull(), yticklabels=False, cbar=False)

Только один признак содержит пропуски - education.

# 2. Подготовим данные для обучения наивной модели.

## Посмотрим как пропуски относятся к общему числу строк в датасете.

In [12]:
count=0
for i in df.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Общее количество строк: ', df.shape[0])
print('Общее количество строк с пропущенными значениями: ', count)
print(f"{(count/df.shape[0])*100}% пропущенных значений.")

Как видно из данных - пропусков крайне мало - меньше 1%.

## Посмотрим на признак с пропусками поближе.

In [13]:
display(df['education'].value_counts().plot.barh())

## Заполним данные самым частым значением

In [14]:
df['education'] = df['education'].fillna(df['education'].mode()[0])

## Удалим признаки, которые не несут смысла и не возможно быстро привести к численному виду.

In [15]:
df = df.drop(['client_id', 'app_date'], axis=1)

## Преобразуем бинарные признаки к виду 0/1.

In [16]:
# выделим бинарные принзнаки
bin_features = ["car", "car_type", "foreign_passport"]
# посмотрим какие значения принимают признаки
for feature in bin_features:
    print(f"Уникальные значения {feature}: {df[feature].unique()}")

## Выделенные признаки не содержат "лишних" значений.

In [17]:
# Преобразуем выделенные признаки
Le = LabelEncoder()

for feature in bin_features:
    Le.fit(df[feature]) 
    df[feature] = Le.transform(df[feature])

# Преобразуем оставшиеся object признаки в dummy переменные.

In [18]:
# выделим object признаки
object_features = df.select_dtypes(include=["object"]).columns
print(object_features)
df = pd.get_dummies(df, columns=object_features)

# 3. Обучим наивную модель.

In [19]:
# отделить test от train по sample
df_train = df.loc[df["sample"] == 0]
# разделим данные
X = df_train.drop(['default', 'sample'], axis = 1)
y = df_train['default'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)


In [20]:
naive_model = LogisticRegression(random_state=RANDOM_SEED, max_iter=1000)
naive_model.fit(X_train, y_train)
y_pred = naive_model.predict(X_test)
y_pred_probs = naive_model.predict_proba(X_test)[:,1]

veiw_model(naive_model, y_test, y_pred)
veiw_metrix(y_test, y_pred, y_pred_probs)

# 3.1. Вывод.
Очевидно, модель очень плохого качества. Объективно - хорошая отправная точка.

# 4. Подробная визуализация
## 4.1. Проведём разведывательный анализ данных.

In [21]:
# снова загрузим данные, для чистоты анализа
df_train = pd.read_csv(DATA_DIR+'train.csv')
df_test = pd.read_csv(DATA_DIR+'test.csv')

In [22]:
# объединим данные в единый датасет, не забыв пометить тестовые данные
df_train["sample"] = 0
df_test['default'] = 0
df_test["sample"] = 1
df = df_train.append(df_test, sort=False).reset_index(drop=True)

In [23]:
pandas_profiling.ProfileReport(df.drop("sample", axis=1))

## Выводы по данным

client_id - идентификатор клиента. Не имеет пропусков. Все значения уникальны. Для нас это значит что результаты модели будут честными. Признак можно удалить. 

app_date - дата подачи заявки. Категориальный признак. Пропусков нет. Записи имеют длину 9 знаков. Это говорит о том, что каждая запись имеет одинаковый формат. Скорее всего данные были сформированы автоматически, следовательно проблем с преобразованием в datetime не должно быть. При обработке можно попробовать выделить самый популярный месяц в новый признак.

education - уровень образования. Категориальный признак. Содержит пропуски, но их меньше 1%. В данной ситуации обработать пропуски будет просто. Признак имеет пять категорий.

sex - пол клиента. Категориальный признак. Пропусков нет. Две категории. Видно что клиентов-женщин больше.

age - возраст заемщика. Числовой признак. Пропусков нет. Выбросы отсутствуют. Данные распределены между 21 и 72 годами. Основная масса данных сосредоточена между 30 и 48 годами.

car - флаг наличия автомобиля. Бинарный признак. Пропуски отсутствуют. Не имеющих автомобиль в два раза больше.

car_type - флаг автомобиля-иномарки. Бинарный признак. Пропуски отсутствуют. Этот признак непрерывно связан с предыдущим - клиенты не имеющие автомобиля вообще, автоматически попадают в категорию не имеющих автомобиль-иномарку. Один из этих признаков можно будет удалить.

decline_app_cnt - количество отказанных прошлых заявок. Числовой признак. Без пропусков. Важнен для обучения модели. Возможно присутствуют выбросы.

good_work - флаг наличия “хорошей” работы. Бинарный признак. Пропуски отсутствуют. Не совсем понятно что означает признак в действительности - отрицательный ответ может означат как работа с низким заработком, так и полное отсутствие работы.

score_bki - скоринговый балл по данным из БКИ. Числовой признак. Без пропусков. Распределение непрерывное, на первый взгляд нормальное относительно медианы. Можно предположить, что данный признак будет очень важен для модели.

bki_request_cnt - количество запросов в БКИ. Числовой признак. Без пропусков. Возможно присутствуют выбросы.

region_rating - рейтинг региона. Числовой признак. Без пропусков. Возможно присутствуют выбросы.

home_address - категоризатор домашнего адреса. Категориальный признак. Пропусков нет. Три категории. Смысловая нагрузка признака не ясна, нужно оценить влияние на модель.

work_address - категоризатор рабочего адреса. Аналогичен предыдущему признаку. 

income - доход заемщика. Числовой признак. Без пропусков. Очень большой разброс. Возможно присутствуют выбросы. Скорее всего признак очень важен.

sna - связь заемщика с клиентами банка. Категориальный признак. Пропусков нет. Четыре категории. Что означает каждая из категорий не ясно. Нужно оценить влияние на модель.

first_time - давность наличия информации о заемщике. Категориальный признак. Пропусков нет. Четыре категории.

foreign_passport - наличие загранпаспорта. Бинарный признак. Пропуски отсутствуют.

## 4.2. Посмотрим корреляцию признаков.

In [24]:
sns.set(rc = {'figure.figsize':(13,11)})
sns.heatmap(df.drop("sample", axis=1).corr(), annot = True, fmt='.2g', cmap='BuPu')

### По данным корреляции:
- видим что home_address и work_address сильно скоррелированы. Оставим home_address.
- присутствует высокая обратная корреляция между тем, новый заёмщик или нет и связью с клиентами банка (признаки sna и first_time), что весьма ожидаемо.
- наблюдается выраженная корреляция между рейтингом региона и сразу тремя признаками: домашний адрес, рабочий адрес и уровень дохода.
- логично присутствует корреляция между количеством отказанных прошлых заявок и двумя признаками: скоринговым баллом БКИ и количеством запросов в БКИ.
- как и ожидалось скоринговый бал БКИ имеет выраженное влияние на целевой признак.

Более подробно оценим корреляцию после преобразования признаков.

### Уберём из данных признак work_address

In [25]:
df = df.drop("work_address", axis=1)

# Иммем 3 группы признаков:
- бинарные - ["sex", "car", "car_type", "good_work", "foreign_passport"]  
- категориальные - ["app_date", "education", "home_address", "sna", "first_time"]  
- числовые - ["age", "decline_app_cnt", "score_bki", "bki_request_cnt", "region_rating", "income"]

## 4.3. Выделим бинарные признаки в список и преобразуем в числовой вид.

In [26]:
bins = ["car", "car_type", "good_work", "foreign_passport"]
Le = LabelEncoder()
for column in bins:
    Le.fit(df[column]) 
    df[column] = Le.transform(df[column])

## 4.4. Поработаем с числовыми признаками.

## age - возраст заемщика.

In [27]:
veiw_numeric(df["age"])

Попробуем прологарифмировать и взять квтадратный корень:

In [28]:
result = sqrt_log_veiw(df['age'])

Квадратный корень признака имеет более однородную выпуклость вверх. Возьмём квадратный корень признака.

In [29]:
df["age"] = result[1]

## decline_app_cnt - количество отказанных прошлых заявок.

In [30]:
veiw_numeric(df["decline_app_cnt"])

Посмотрим выбросы:

In [31]:
outliers = outliers_iqr((df['decline_app_cnt']))

Метот обнаружил очень много выбросов. Заметим что за выброс метод посчитал любое значение >0.

Посмотрим распределение логарифма и квадратного корня признака:

In [32]:
result = sqrt_log_veiw(df['decline_app_cnt'])

Избавиться от большого перепада не выходит. Возьмём логарифм признака чтобы уменьшить масштаб.

In [33]:
decline_app_cnt = result[0]

## score_bki - скоринговый балл по данным из БКИ.

In [34]:
veiw_numeric(df["score_bki"])

Распределение нормальное. Сдвинем на медиану вправо и больше не будем ничего менять.

In [35]:
# df["score_bki"] = (df["score_bki"] - df["score_bki"].median())

## bki_request_cnt - количество запросов в БКИ.

In [36]:
veiw_numeric(df["bki_request_cnt"])

Посмотрим выбросы:

In [37]:
outliers = outliers_iqr((df['bki_request_cnt']))

In [38]:
veiw_numeric(df['bki_request_cnt'].drop(outliers.index))

Выбросов довольно много, но распределение стало лучше.

Посмотрим распределение логарифма и квадратного корня признака:

In [39]:
result = sqrt_log_veiw(df['bki_request_cnt'])

Не будем терять почти 3 тысячи данных. Оставим квадратный корень признака.

In [40]:
df["bki_request_cnt"] = result[1]

## region_rating - рейтинг региона.

In [41]:
veiw_numeric(df["region_rating"])

Посмотрим распределение логарифма и квадратного корня признака:

In [42]:
result = sqrt_log_veiw(df['region_rating'])

Распределение практически не реагирует на манипуляции с данными. Оставим признак в изначальном виде.

## income - доход заемщика.

In [43]:
veiw_numeric(df["income"])

In [44]:
outliers = outliers_iqr(df["income"])
veiw_numeric(df.drop(outliers.index)["income"])

In [45]:
result = sqrt_log_veiw(df['income'])

Выбросов очень много, а логарифмирование даёт хороший результат. Его и оставим.

In [46]:
df["income"] = result[0]

## 4.5. Поработаем с категориальными признаками.

## app_date - дата подачи заявки. 

Приведём к datetime.

In [47]:
df['app_date'] = pd.to_datetime(df['app_date'])

Посмотрим общщее распределение.

In [48]:
df["app_date"].value_counts().plot(kind='bar', figsize=(17,13), fontsize=9)

Посмотрим распределение по месяцам.

In [49]:
month = df['app_date'].apply(lambda x: x.month)
month.value_counts().plot(kind='bar')

Имеем распределение по 4 месяцам. Можно выделить месяцы в новый признак month, а так же выделить дамми-переменные.

Посмотрим на распределение по дням недели.

In [50]:
weekday = df['app_date'].apply(lambda x: x.weekday())
weekday.value_counts().plot(kind='bar')

Распределение по дням недели выделим в новый признак weekday.

# education
Образование клиента.  
Посмотрим какие значения принимает признак.  

In [51]:
print(df['education'].unique())
df['education'].value_counts().plot(kind='bar')

Количество пропусков, как было выяснено ранее, не велико. Заполним их самым частым значением.

In [52]:
# заполняем самым частым значением
df['education'] = df['education'].fillna(df['education'].mode()[0])
df['education'].value_counts().plot(kind='bar')

In [53]:
# Le = LabelEncoder()
Le.fit(df['education']) 
df['education'] = Le.transform(df['education'])

# 5. Сгенерируем новые признаки.

In [54]:
# Добавим новые признаки weekday и month, и удалим app_date
df['month'] = month
df['weekday'] = weekday
df = df.drop('app_date', axis=1)

In [55]:
# Выделим бинарный признак, где 0 - нет отказов, >0 - есть отказы на этапе проектирования признаков.
df['decline'] = df['decline_app_cnt'].apply(lambda x: 0 if x == 0 else 1)
# запишем в df['decline_app_cnt'] ранее полученные результаты обработки
df['decline_app_cnt'] = decline_app_cnt

In [56]:
# sex получим дамми-переменные и удалим прзнак
df = pd.get_dummies(df, columns=['sex'])

In [57]:
default = df["default"].copy()
df = df.drop("default", axis=1)
df.insert(0, "default", default)

## Повторно посмотрим корреляцию.

In [58]:
sns.set(rc = {'figure.figsize':(19,11)})
sns.heatmap(df.drop(["sample", "client_id"], axis=1).corr(), annot = True, fmt='.1g', linewidths = 1, cmap='coolwarm')

Удалим признак car_type из-за высокой корреляции с признаком car и отсутствием сильного влияния на дефолт заёмщика.

In [59]:
df = df.drop('car_type', axis=1)

Созданный признак decline не несёт новой информации, следовательно можно не включать его в итоговый набор.

In [60]:
df = df.drop('decline', axis=1)

Удалим признак sna из-за высокой корреляции с принаком first_time и более существенным влиянием на дефолт.

In [61]:
df = df.drop('sna', axis=1)

# 6. Обучение модели.

In [62]:
df_train = df.loc[df["sample"] == 0].drop("sample", axis=1)

## Модель 1.

In [63]:
X = df_train.drop(['client_id', 'default'], axis = 1)
y = df_train['default'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

model_1 = LogisticRegression(random_state=RANDOM_SEED, max_iter=5000)
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)
y_pred_probs = model_1.predict_proba(X_test)[:,1]

veiw_model(model_1, y_test, y_pred)
veiw_metrix(y_test, y_pred, y_pred_probs)

Не смотря на высокий roc_aur модель всё ещё весьма не качественная.  
Попробуем преобразовать категориальные признаки в dummy-переменные.

In [64]:
new_df = df.copy()
categorical = ["education", "home_address", "first_time", "month", "weekday"]
new_df = pd.get_dummies(new_df, columns=categorical)

In [65]:
df_train = new_df.loc[new_df["sample"] == 0].drop("sample", axis=1)

## Модель 2.

In [66]:
X = df_train.drop(['client_id', 'default'], axis = 1)
y = df_train['default'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

model_2 = LogisticRegression(random_state=RANDOM_SEED, max_iter=5000)
model_2.fit(X_train, y_train)
y_pred = model_2.predict(X_test)
y_pred_probs = model_2.predict_proba(X_test)[:,1]

veiw_model(model_2, y_test, y_pred)
veiw_metrix(y_test, y_pred, y_pred_probs)

Модель улучшилась, но очень слабо.  
Попробуем обучить модель на отбалансированных признаках.

## Балансируем признаки.
## Модель 3.

In [67]:
model_3 = LogisticRegression(class_weight='balanced', random_state=RANDOM_SEED, max_iter=5000)
model_3.fit(X_train, y_train)
y_pred = model_3.predict(X_test)
y_pred_probs = model_3.predict_proba(X_test)[:,1]

veiw_model(model_3, y_test, y_pred)
veiw_metrix(y_test, y_pred, y_pred_probs)

Модель стала лучше находить дефолтных клиентов. Не смотря на то, что появилось много ошибок второго рода, число ошибок первого рода резко сократилось, что, несомненно, лучше.  
Попробуем подобрать гипер параметры чтобы ещё улучшить модель.

## Регуляризация

In [68]:
param_grid = [{'C': np.logspace(-4.0, 4.0, num=20)}]

gridsearch = GridSearchCV(model_3, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
model_4 = gridsearch.best_estimator_

best_parameters = model_4.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

y_pred = model_4.predict(X_test)
y_pred_probs = model_4.predict_proba(X_test)[:,1]

veiw_model(model_4, y_test, y_pred)
veiw_metrix(y_test, y_pred, y_pred_probs)

Обучим модель с выявленными гиперпараметрами.

In [69]:
model_4 = LogisticRegression(C=0.004832930238571752, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, \
                    l1_ratio=None, max_iter=5000, multi_class='auto', n_jobs=None, penalty='l2', \
                    random_state=RANDOM_SEED, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
model_4.fit(X_train, y_train)

y_pred_probs = model_4.predict_proba(X_test)[:,1]

Регуляризация позволила улучшить результаты модели.  
Не смотря на результаты, модель ещё слаба, но улучшить её при текущих знаниях не представляется возможным.
В целом, результат удовлетворительный, можно фиксировать его в соревновании.

# 7. Запуск модели на новых данных и Сабмит.

In [70]:
df_test = new_df.loc[new_df['sample'] == 1].drop(['sample', 'default'], axis=1)
default = model_4.predict_proba(df_test.drop('client_id', axis=1))[:,1]

df_test['default'] = default

submission = df_test[['client_id', 'default']]
submission.to_csv('submission.csv', index=False)

# Выводы.
Модель справляется с поставленной задачей примерно на 68% что весьма много. Возможно стоит поработать над обработкой данных, возможно нужно лучше подбирать гиперпараметры, возможно нужно найти больше дополнительных признаков. В любом случае Результат удовлетворительный и мне есть куда расти, как исследователю.